# <center> Предсказание пола клиента </center>

### Необходимо выявить пол клиента, основываясь на его транзакционных исторических данных. В роли метрики качества выступает [ROC AUC](https://dyakonov.org/2017/07/28/auc-roc-%D0%BF%D0%BB%D0%BE%D1%89%D0%B0%D0%B4%D1%8C-%D0%BF%D0%BE%D0%B4-%D0%BA%D1%80%D0%B8%D0%B2%D0%BE%D0%B9-%D0%BE%D1%88%D0%B8%D0%B1%D0%BE%D0%BA/), который и нужно будет максимизировать.

## Описание файлов
- transactions.csv - исторические транзакции банковских клиентов
- gender.csv - информация по полу для части клиентов (null - для тестовых)
- tr_mcc_codes.csv - mcc-коды транзакций
- tr_types.csv - типы транзакций

## Описание полей
### transactions.csv
- customer_id - идентификатор клиента
- tr_datetime - день и время совершения транзакции (дни нумеруются с начала данных)
- mcc_code - mcc-код транзакции
- tr_type - тип транзакции
- amount - сумма транзакции в условных единицах; со знаком "+" — начисление средств клиенту, "-" — списание средств
- term_id - идентификатор терминала

### gender.csv
- customer_id - идентификатор клиента
- gender - пол клиента (пустые значения - тестовые клиенты)

### tr_mcc_codes.csv
- mcc_code - mcc-код транзакции
- mcc_description - описание mcc-кода транзакции

### tr_types.csv
- tr_type - тип транзакции
- tr_description - описание типа транзакции

## Задачи:
- Разработать модель бинарной классификации для определения пола клиента. Никаких ограничений к модели - может быть что угодно от KNN до трансформеров. Главное, чтобы ROC AUC на отложенном тесте получился выше 77.5%.
- Интерпретировать результаты модели: важность входящих в нее переменных, демонстрация на нескольких примерах, почему получился соответствующий прогноз. Последнее позволит понять, какой пол к какому из таргетов (0/1) принадлежит. Опять же, полная свобода выбора подходов! Полезные ключевые слова: gain, permutation importance, SHAP. 
- Конвертировать результаты в отчет без кода (идеально - напрямую в [html](https://stackoverflow.com/questions/49907455/hide-code-when-exporting-jupyter-notebook-to-html))

#### P.S. Не забываем про [PEP8](https://www.python.org/dev/peps/pep-0008/)!

In [1]:
import numpy as np
import pandas as pd

In [2]:
tr_mcc_codes = pd.read_csv("data/tr_mcc_codes.csv", sep=";", index_col="mcc_code")
tr_types = pd.read_csv("data/tr_types.csv", sep=";", index_col="tr_type")

transactions = pd.read_csv("data/transactions.csv", index_col="customer_id")
gender = pd.read_csv("data/gender.csv", index_col="customer_id")

In [3]:
# TODO
tr_mcc_codes.head(5)


,mcc_description
mcc_code,
742,Ветеринарные услуги
1711,"Генеральные подрядчики по вентиляции, теплосна..."
1731,Подрядчики по электричеству
1799,"Подрядчики, специализированная торговля — нигд..."
2741,Разнообразные издательства/печатное дело


In [4]:
tr_types.head(5)

,tr_description
tr_type,
3200,Плата за предоставление услуг посредством моби...
3210,Плата за предоставление отчета по счету карты ...
3800,Плата за обслуживание банковской карты (за пер...
4000,Плата за получение наличных в Сбербанке
4001,Плата за получение наличных в Сбербанке (в дру...


In [5]:
transactions.head(5)

,tr_datetime,mcc_code,tr_type,amount,term_id
customer_id,,,,,
39026145,0 10:23:26,4814,1030,-2245.92,NaN
39026145,1 10:19:29,6011,7010,56147.89,NaN
39026145,1 10:20:56,4829,2330,-56147.89,NaN
39026145,1 10:39:54,5499,1010,-1392.47,NaN
39026145,2 15:33:42,5499,1010,-920.83,NaN


In [6]:
gender.head(5)

,gender
customer_id,
6740969,1.0
36599781,0.0
91453656,1.0
17431962,1.0
72568668,1.0


In [7]:
gender.isna().sum()

gender    3600
dtype: int64

In [8]:
# 1. Объединяем транзакции с полом
df = transactions.merge(gender, left_index=True, right_index=True, how='inner')

# 2. Присоединяем описания MCC
df = df.merge(tr_mcc_codes, on='mcc_code', how='left')

# 3. Считаем самые частые MCC для каждой группы
top_mcc_by_gender = (
    df.groupby('gender')['mcc_description']
      .apply(lambda x: x.value_counts().head(15))
)

print(top_mcc_by_gender)


gender                                                                                                                                                            
0.0     Финансовые институты — снятие наличности автоматически                                                                                                        409128
        Финансовые институты — снятие наличности вручную                                                                                                              335588
        Звонки с использованием телефонов, считывающих магнитную ленту                                                                                                324566
        Бакалейные магазины, супермаркеты                                                                                                                             275969
        Денежные переводы                                                                                                                        

In [9]:
df['abs_amount'] = df['amount'].abs()
df.groupby('gender')['abs_amount'].mean()



gender
0.0     58521.592756
1.0    103270.402645
Name: abs_amount, dtype: float64

1.0 - мужчины <br>
0.0 - женщины <br>
<h>По видами операций и общим тратам 

In [10]:

# Берём копию, чтобы не менять исходные
tr = transactions.copy()

# Признак: абсолютная сумма (удобно для статистик)
tr["abs_amount"] = tr["amount"].abs()

# Группируем по клиенту и считаем базовые агрегаты
agg_money = tr.groupby("customer_id").agg({
    "amount": ["mean", "std", "sum", "min", "max"],
    "abs_amount": ["mean", "std", "max"],
})

# Многоуровневый индекс колонок → делаем плоские имена
agg_money.columns = ["_".join(col) for col in agg_money.columns]
agg_money = agg_money.fillna(0)


In [11]:

agg_money.head(5)

,amount_mean,amount_std,amount_sum,amount_min,amount_max,abs_amount_mean,abs_amount_std,abs_amount_max
customer_id,,,,,,,,
6815,-5528.561372,168370.985787,-1249454.87,-224591.58,2470507.35,29480.358717,165850.943654,2470507.35
22899,-2691.438419,127851.642622,-629796.59,-868607.92,875907.15,52425.823462,116589.255466,875907.15
27914,-26820.061441,104675.148673,-2977026.82,-462658.65,89836.63,34205.281261,102479.773798,462658.65
28753,-90293.479116,785586.199773,-26546282.86,-3244539.76,3099363.77,401228.234014,681023.410567,3244539.76
31385,-5972.612548,33418.557369,-2180003.58,-364366.15,199886.50,12829.239671,31424.966482,364366.15


In [12]:
# Фильтруем по знаку транзакции
tr["is_income"] = tr["amount"] > 0

agg_sign = tr.groupby(["customer_id", "is_income"])["amount"].agg(["count", "sum"]).unstack().fillna(0)
agg_sign.columns = ["ops_expense", "ops_income", "sum_expense", "sum_income"]

# Соединяем с базовыми фичами
features = agg_money.join(agg_sign, how="left").fillna(0)

# Добавим отношение приходов к расходам
features["income_to_expense_ratio"] = (features["sum_income"].abs() + 1) / (features["sum_expense"].abs() + 1)


In [13]:
features.head(10)

,amount_mean,amount_std,amount_sum,amount_min,amount_max,abs_amount_mean,abs_amount_std,abs_amount_max,ops_expense,ops_income,sum_expense,sum_income,income_to_expense_ratio
customer_id,,,,,,,,,,,,,
6815,-5528.561372,168370.985787,-1249454.87,-224591.58,2470507.35,29480.358717,165850.943654,2470507.35,220.0,6.0,-3956007.97,2706553.10,0.684163
22899,-2691.438419,127851.642622,-629796.59,-868607.92,875907.15,52425.823462,116589.255466,875907.15,166.0,68.0,-6448719.64,5818923.05,0.902338
27914,-26820.061441,104675.148673,-2977026.82,-462658.65,89836.63,34205.281261,102479.773798,462658.65,76.0,35.0,-3386906.52,409879.70,0.121019
28753,-90293.479116,785586.199773,-26546282.86,-3244539.76,3099363.77,401228.234014,681023.410567,3244539.76,244.0,50.0,-72253691.83,45707408.97,0.632596
31385,-5972.612548,33418.557369,-2180003.58,-364366.15,199886.50,12829.239671,31424.966482,364366.15,342.0,23.0,-3431338.03,1251334.45,0.364678
38084,-2069.954916,138982.400359,-1970597.08,-1142759.68,1235253.67,57453.247227,126554.534724,1235253.67,838.0,114.0,-28333044.22,26362447.14,0.930449
42096,-5907.505855,63639.188660,-5529425.48,-337785.73,673774.73,28587.716581,57155.495552,673774.73,870.0,66.0,-16143764.10,10614338.62,0.657488
49101,-38776.769231,134367.931805,-2016392.00,-449183.15,449183.15,78857.727692,115097.300693,449183.15,45.0,7.0,-3058496.92,1042104.92,0.340725
49793,-47815.367700,161784.403694,-24529283.63,-673774.73,112295.79,50205.757680,161057.216319,673774.73,488.0,25.0,-25142418.66,613135.03,0.024387


In [14]:
X = features.join(gender, how="inner")
print(X.shape)
X.head()


(12000, 14)


,amount_mean,amount_std,amount_sum,amount_min,amount_max,abs_amount_mean,abs_amount_std,abs_amount_max,ops_expense,ops_income,sum_expense,sum_income,income_to_expense_ratio,gender
customer_id,,,,,,,,,,,,,,
6815,-5528.561372,168370.985787,-1249454.87,-224591.58,2470507.35,29480.358717,165850.943654,2470507.35,220.0,6.0,-3956007.97,2706553.10,0.684163,NaN
22899,-2691.438419,127851.642622,-629796.59,-868607.92,875907.15,52425.823462,116589.255466,875907.15,166.0,68.0,-6448719.64,5818923.05,0.902338,1.0
27914,-26820.061441,104675.148673,-2977026.82,-462658.65,89836.63,34205.281261,102479.773798,462658.65,76.0,35.0,-3386906.52,409879.70,0.121019,1.0
28753,-90293.479116,785586.199773,-26546282.86,-3244539.76,3099363.77,401228.234014,681023.410567,3244539.76,244.0,50.0,-72253691.83,45707408.97,0.632596,0.0
31385,-5972.612548,33418.557369,-2180003.58,-364366.15,199886.50,12829.239671,31424.966482,364366.15,342.0,23.0,-3431338.03,1251334.45,0.364678,0.0


In [15]:
# Берём только клиентов с известным полом
df = transactions.join(gender, how="inner")

# Уникальность MCC
agg_mcc = (
    df.groupby("customer_id")["mcc_code"]
      .agg([
          ("mcc_unique_count", "nunique"),     # сколько разных категорий MCC у клиента
          ("mcc_total_count", "count")         # всего транзакций
      ])
)

# Отношение: разнообразие MCC (чем выше, тем более "разнонаправленные" траты)
agg_mcc["mcc_diversity_ratio"] = agg_mcc["mcc_unique_count"] / agg_mcc["mcc_total_count"]


In [16]:
agg_mcc.head(10)


,mcc_unique_count,mcc_total_count,mcc_diversity_ratio
customer_id,,,
6815,6,226,0.026549
22899,11,234,0.047009
27914,6,111,0.054054
28753,30,294,0.102041
31385,17,365,0.046575
38084,37,952,0.038866
42096,39,936,0.041667
49793,16,513,0.031189
50940,7,118,0.059322


In [17]:
df['mcc_code']

customer_id
39026145    4814
39026145    6011
39026145    4829
39026145    5499
39026145    5499
            ... 
61870738    5499
61870738    5411
61870738    5499
61870738    5541
61870738    6011
Name: mcc_code, Length: 5393337, dtype: int64

In [18]:
# Топ-N самых частых MCC в выборке
top_mcc = (
    df["mcc_code"].value_counts()
      .head(20)
      .index.tolist()
)

# Для каждого MCC — доля транзакций клиента с этим кодом
mcc_dummies = (
    pd.get_dummies(df["mcc_code"])
      .groupby(df.index)
      .sum()
)

# Оставляем только топовые MCC
mcc_dummies = mcc_dummies[top_mcc]

# Переводим в доли
mcc_dummies = mcc_dummies.div(mcc_dummies.sum(axis=1), axis=0).fillna(0)

# Переименовываем колонки для читаемости
mcc_dummies.columns = [f"mcc_{code}_share" for code in mcc_dummies.columns]

# Соединяем с фичами
features = features.join(agg_mcc, how="left").join(mcc_dummies, how="left").fillna(0)


In [19]:
features.head(5)

,amount_mean,amount_std,amount_sum,amount_min,amount_max,abs_amount_mean,abs_amount_std,abs_amount_max,ops_expense,ops_income,...,mcc_5814_share,mcc_5921_share,mcc_5999_share,mcc_5311_share,mcc_5977_share,mcc_5964_share,mcc_5983_share,mcc_5691_share,mcc_5211_share,mcc_8999_share
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,-5528.561372,168370.985787,-1249454.87,-224591.58,2470507.35,29480.358717,165850.943654,2470507.35,220.0,6.0,...,0.000000,0.00885,0.000000,0.128319,0.000000,0.0,0.0,0.000000,0.000000,0.000000
22899,-2691.438419,127851.642622,-629796.59,-868607.92,875907.15,52425.823462,116589.255466,875907.15,166.0,68.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
27914,-26820.061441,104675.148673,-2977026.82,-462658.65,89836.63,34205.281261,102479.773798,462658.65,76.0,35.0,...,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
28753,-90293.479116,785586.199773,-26546282.86,-3244539.76,3099363.77,401228.234014,681023.410567,3244539.76,244.0,50.0,...,0.204545,0.00000,0.049242,0.003788,0.003788,0.0,0.0,0.007576,0.007576,0.007576
31385,-5972.612548,33418.557369,-2180003.58,-364366.15,199886.50,12829.239671,31424.966482,364366.15,342.0,23.0,...,0.002849,0.00000,0.019943,0.000000,0.002849,0.0,0.0,0.011396,0.000000,0.000000


In [20]:
tr_time = transactions.copy()

# Преобразуем и удаляем мусор
tr_time["tr_datetime"] = pd.to_datetime(tr_time["tr_datetime"], errors="coerce", dayfirst=True)
tr_time = tr_time.dropna(subset=["tr_datetime"])

# Извлекаем фичи
tr_time["hour"] = tr_time["tr_datetime"].dt.hour
tr_time["weekday"] = tr_time["tr_datetime"].dt.dayofweek



/var/folders/m8/xc5fwxf55gl143x92mz7q4dc0000gn/T/ipykernel_82955/1902843223.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tr_time["tr_datetime"] = pd.to_datetime(tr_time["tr_datetime"], errors="coerce", dayfirst=True)


In [21]:
tr_time.head(10)

,tr_datetime,mcc_code,tr_type,amount,term_id,hour,weekday
customer_id,,,,,,,
39026145,2032-01-01 16:16:19,5499,1010,-4238.04,NaN,16,3
39026145,2032-01-01 16:41:33,5499,1010,-1909.03,NaN,16,3
39026145,2033-01-01 10:06:45,4814,1030,-2245.92,NaN,10,5
39026145,2033-01-01 12:13:11,5499,1010,-6378.40,NaN,12,5
39026145,2033-01-01 12:14:55,5499,1010,-44.92,NaN,12,5
39026145,2035-01-01 16:05:29,4814,1030,-1122.96,NaN,16,0
39026145,2037-01-01 11:36:27,6011,7010,22459.16,NaN,11,3
39026145,2037-01-01 11:38:38,4814,1030,-3368.87,NaN,11,3
39026145,2037-01-01 11:41:24,6011,2010,-15721.41,NaN,11,3


In [22]:
# one-hot кодировка часов
hour_dummies = pd.get_dummies(tr_time["hour"], prefix="hour")

# группируем по клиенту, суммируем
hour_agg = hour_dummies.groupby(tr_time.index).sum()

# превращаем в доли
hour_agg = hour_agg.div(hour_agg.sum(axis=1), axis=0).fillna(0)


In [23]:
hour_dummies.head(1)

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
customer_id,,,,,,,,,,,,,,,,,,,,,
39026145,False,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [24]:
hour_agg.head(1)

,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,...,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,0.0,0.0,0.029412,0.0,0.0,0.0,0.029412,0.029412,0.0,0.117647,...,0.088235,0.088235,0.0,0.029412,0.029412,0.058824,0.058824,0.029412,0.0,0.0


In [25]:
weekday_dummies = pd.get_dummies(tr_time["weekday"], prefix="wd")
weekday_agg = weekday_dummies.groupby(tr_time.index).sum()
weekday_agg = weekday_agg.div(weekday_agg.sum(axis=1), axis=0).fillna(0)


In [26]:
# Ночь = 23:00–05:59
tr_time["is_night"] = tr_time["hour"].between(23, 23) | tr_time["hour"].between(0, 5)

night_ratio = tr_time.groupby(tr_time.index)["is_night"].mean()


In [27]:
tr_time["is_weekend"] = tr_time["weekday"] >= 5
weekend_ratio = tr_time.groupby(tr_time.index)["is_weekend"].mean()


In [28]:
features = (
    features
    .join(hour_agg, how="left")
    .join(weekday_agg, how="left")
    .join(night_ratio.rename("night_ratio"), how="left")
    .join(weekend_ratio.rename("weekend_ratio"), how="left")
    .fillna(0)
)


Проверям фичи на легит 

In [29]:
print("Total rows:", features.shape[0])
print("Rows with any NaN:", features.isna().any(axis=1).sum())

# Посмотреть, какие именно NaN
nan_cols = features.columns[features.isna().any()]
nan_cols


Total rows: 15000
Rows with any NaN: 0


Index([], dtype='object')

In [30]:
const_cols = [col for col in features.columns if features[col].nunique() <= 1]
const_cols


[]

In [31]:
np.isinf(features.values).sum(), np.isnan(features.values).sum()


(np.int64(0), np.int64(0))

In [32]:
features.describe(percentiles=[0.01, 0.99]).T


,count,mean,std,min,1%,50%,99%,max
amount_mean,15000.0,-3.266016e+04,8.133880e+04,-2.093564e+06,-3.152279e+05,-1.605239e+04,3.549439e+04,1.385521e+06
amount_std,15000.0,1.851879e+05,3.552660e+05,0.000000e+00,1.475548e+04,1.056684e+05,1.264732e+06,1.701161e+07
amount_sum,15000.0,-9.380339e+06,3.311783e+07,-1.505729e+09,-8.610180e+07,-4.856003e+06,9.624127e+06,6.289847e+08
amount_min,15000.0,-1.282857e+06,3.254494e+06,-1.459845e+08,-1.000567e+07,-6.737747e+05,-5.165606e+04,1.010662e+06
amount_max,15000.0,1.335665e+06,3.667066e+06,-5.614789e+05,-4.495551e+03,5.165606e+05,1.122958e+07,1.459845e+08
...,...,...,...,...,...,...,...,...
wd_4,15000.0,1.291900e-01,1.033064e-01,0.000000e+00,0.000000e+00,1.250000e-01,5.000000e-01,1.000000e+00
wd_5,15000.0,1.203515e-01,9.700962e-02,0.000000e+00,0.000000e+00,1.176471e-01,4.285714e-01,1.000000e+00
wd_6,15000.0,1.464188e-01,1.086554e-01,0.000000e+00,0.000000e+00,1.447368e-01,5.000000e-01,1.000000e+00
night_ratio,15000.0,1.411378e-01,1.716110e-01,0.000000e+00,0.000000e+00,7.692308e-02,6.885261e-01,1.000000e+00


In [33]:
agg_terms = (
    tr.groupby("customer_id")["term_id"]
      .agg([
          ("term_unique_count", "nunique"),
          ("term_total_count", "count"),
      ])
)

agg_terms["term_diversity_ratio"] = (
    agg_terms["term_unique_count"] / agg_terms["term_total_count"]
)


In [34]:
tr.head(10)

,tr_datetime,mcc_code,tr_type,amount,term_id,abs_amount,is_income
customer_id,,,,,,,
39026145,0 10:23:26,4814,1030,-2245.92,NaN,2245.92,False
39026145,1 10:19:29,6011,7010,56147.89,NaN,56147.89,True
39026145,1 10:20:56,4829,2330,-56147.89,NaN,56147.89,False
39026145,1 10:39:54,5499,1010,-1392.47,NaN,1392.47,False
39026145,2 15:33:42,5499,1010,-920.83,NaN,920.83,False
39026145,2 15:53:49,5541,1010,-14643.37,NaN,14643.37,False
39026145,3 15:29:08,5499,1010,-1010.66,NaN,1010.66,False
39026145,4 12:11:57,5200,1010,-2829.85,NaN,2829.85,False
39026145,5 15:19:19,5499,1010,-628.86,NaN,628.86,False


In [35]:
tr["tr_datetime"] = pd.to_datetime(tr["tr_datetime"], errors="coerce")
#tr = tr.dropna(subset=["tr_datetime"])

tr["date"] = tr["tr_datetime"].dt.date

agg_days = (
    tr.groupby("customer_id")["date"]
      .agg(active_days="nunique", total_ops="count")
)
agg_days["ops_per_active_day"] = agg_days["total_ops"] / agg_days["active_days"]


/var/folders/m8/xc5fwxf55gl143x92mz7q4dc0000gn/T/ipykernel_82955/2409377309.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tr["tr_datetime"] = pd.to_datetime(tr["tr_datetime"], errors="coerce")


In [36]:
features = (
    features
    .join(agg_terms, how="left")
    .join(agg_days, how="left")
    .fillna(0)
)


In [37]:
# Берём только клиентов, у которых известен пол
X = features.join(gender, how="inner")

y = X["gender"]
X = X.drop(columns=["gender"])


In [38]:
X.shape, y.shape


((12000, 75), (12000,))

In [39]:

print("Rows with any NaN:", y.isna().sum())


Rows with any NaN: 3600


In [40]:
# Индексы, где y == NaN
nan_idx = y[y.isna()].index

# Выведем соответствующие строки из X
X.loc[nan_idx]



,amount_mean,amount_std,amount_sum,amount_min,amount_max,abs_amount_mean,abs_amount_std,abs_amount_max,ops_expense,ops_income,...,wd_5,wd_6,night_ratio,weekend_ratio,term_unique_count,term_total_count,term_diversity_ratio,active_days,total_ops,ops_per_active_day
customer_id,,,,,,,,,,,,,,,,,,,,,
6815,-5528.561372,1.683710e+05,-1249454.87,-224591.58,2470507.35,29480.358717,165850.943654,2470507.35,220.0,6.0,...,0.058824,0.117647,0.029412,0.176471,42,136,0.308824,23,34,1.478261
49793,-47815.367700,1.617844e+05,-24529283.63,-673774.73,112295.79,50205.757680,161057.216319,673774.73,488.0,25.0,...,0.150000,0.150000,0.050000,0.300000,90,281,0.320285,40,60,1.500000
85327,-6599.337674,2.789175e+04,-567543.04,-224591.58,33688.74,7931.218140,27538.504122,224591.58,84.0,2.0,...,0.333333,0.000000,0.000000,0.333333,18,50,0.360000,9,9,1.000000
186730,-11567.638744,3.545322e+05,-2579583.44,-1122957.89,2245915.77,167363.694081,312555.085034,2245915.77,183.0,40.0,...,0.125000,0.000000,0.062500,0.125000,41,149,0.275168,18,32,1.777778
308038,-12090.699703,1.877463e+05,-6915880.23,-2695098.93,2695098.93,45253.845122,182602.364024,2695098.93,512.0,60.0,...,0.090909,0.236364,0.109091,0.327273,122,361,0.337950,38,55,1.447368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99915912,-9180.910234,3.461633e+04,-10613132.23,-444579.03,134754.95,14188.814645,32880.929853,444579.03,1071.0,85.0,...,0.097744,0.172932,0.090226,0.270677,158,706,0.223796,47,133,2.829787
99917144,-13950.857149,5.225313e+04,-9347074.29,-527790.21,22459.16,14192.209328,52187.997535,527790.21,663.0,7.0,...,0.117647,0.161765,0.279412,0.279412,39,436,0.089450,36,68,1.888889
99938948,-9969.220442,4.370500e+04,-6320485.76,-426724.00,415494.42,17768.428864,41151.595395,426724.00,591.0,43.0,...,0.135135,0.189189,0.243243,0.324324,162,390,0.415385,38,74,1.947368


Много Nan в поле gender 

In [41]:
nan_idx = y[y.isna()].index
X_clean = X.drop(nan_idx)
y_clean = y.drop(nan_idx)

print("Размер после удаления:", len(y_clean))
print(y_clean.value_counts(dropna=False))


Размер после удаления: 8400
gender
0.0    4651
1.0    3749
Name: count, dtype: int64


проверили балан классов, перед очситкой, можем удалять

In [42]:
# Удаляем строки, где y = NaN
nan_idx = y[y.isna()].index
X = X.drop(nan_idx)
y = y.drop(nan_idx)

# Синхронизируем индексы
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)


In [43]:
X.shape, y.shape

((8400, 75), (8400,))

In [44]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=42
)

for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    print(f"Fold {fold}: train={len(train_idx)}, valid={len(valid_idx)}")


Fold 0: train=6720, valid=1680
Fold 1: train=6720, valid=1680
Fold 2: train=6720, valid=1680
Fold 3: train=6720, valid=1680
Fold 4: train=6720, valid=1680


In [45]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.05,
    depth=8,
    loss_function="Logloss",
    eval_metric="AUC",
    random_seed=42,
    verbose=False
)


In [46]:
from sklearn.metrics import roc_auc_score

for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    model.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=False)

    preds = model.predict_proba(X_valid)[:, 1]

    fold_auc = roc_auc_score(y_valid, preds)
    print(f"Fold {fold} AUC: {fold_auc:.4f}")


Fold 0 AUC: 0.8253
Fold 1 AUC: 0.8341
Fold 2 AUC: 0.8222
Fold 3 AUC: 0.8385
Fold 4 AUC: 0.8286


In [47]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_aucs = []

for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

    model = CatBoostClassifier(
        iterations=2000,
        learning_rate=0.05,
        depth=8,
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=False
    )
    
    model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

    preds = model.predict_proba(X_valid)[:, 1]
    fold_auc = roc_auc_score(y_valid, preds)
    fold_aucs.append(fold_auc)

    print(f"Fold {fold} AUC: {fold_auc:.4f}")

print("\nMean AUC:", sum(fold_aucs) / len(fold_aucs))


Fold 0 AUC: 0.8253
Fold 1 AUC: 0.8341
Fold 2 AUC: 0.8222
Fold 3 AUC: 0.8385
Fold 4 AUC: 0.8286

Mean AUC: 0.8297427896201386
